# Setup

### 1. Datasets

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

* trade_details: dataset original con datos reales de operaciones financieras.
* trade_details_snapshot: copia de seguridad por posibles perdidas de datos.

### 2. Columnas y significado:

* mfamily: indica la familia de operaciones a la que pertenece.
* mgroup: indica el grupo de operaciones dentro de la familia.
* mtype: indica el tipo de operación dentro del grupo.
* origin_trade_number: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
* origin_contract_number: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
* maturity: fecha de finalización del contrato de cada operación.

### 3. Descripción del problema:

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg* 

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

### 4. Reto:

* Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
* Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
* Hacerlo de la manera más eficiente posible a nivel computacional.

## imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
# import tensorflow as tf
# import keras
import statsmodels.api as sm
import scipy.stats as stats
import datetime
import warnings
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from IPython.display import display
from sklearn.preprocessing import StandardScaler, RobustScaler
from tqdm.auto import tqdm
import mapply
import warnings
import importlib

/home/rockhight/Documents/gitrepos/formacion_python/pandas/Ejercicio Pandas 1/envi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mapply.init(n_workers = -1)

In [3]:
%matplotlib inline

In [4]:
warnings.filterwarnings('ignore')

In [5]:
trade_details = pd.read_csv("trade_details.csv",sep = ';')
trade_details_snapshot = pd.read_csv("trade_details_snapshot.csv",sep = ';')
display(trade_details.head())
display(trade_details_snapshot.head())
display(trade_details.info())
display(trade_details_snapshot.info())

,mfamily,mgroup,mtype,origin_trade_number,origin_contract_number,maturity
0,IRD,BOND,NaN,316391872,678876251,2021-09-22
1,CURR,FXD,FXD,32734782,54853428,2021-09-22
2,IRD,LN_BR,NaN,1111,2222,2022-10-06
3,IRD,IRS,NaN,2222222,2222222,2024-10-15
4,SCF,SCF,SCF,3815982,3672136,NaN


,mfamily,mgroup,mtype,origin_trade_number,origin_contract_number,maturity
0,CURR,FXD,SWLEG,19665185,18622136,2020-04-29
1,CURR,FXD,SWLEG,19665186,18622136,2020-12-30
2,CURR,FXD,SWLEG,19772399,18724280,2020-11-05
3,CURR,FXD,SWLEG,19772400,18724280,2021-11-05
4,CURR,FXD,SWLEG,20980932,19883451,2020-02-02


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   mfamily                 31 non-null     object
 1   mgroup                  31 non-null     object
 2   mtype                   14 non-null     object
 3   origin_trade_number     31 non-null     int64 
 4   origin_contract_number  31 non-null     int64 
 5   maturity                22 non-null     object
dtypes: int64(2), object(4)
memory usage: 1.6+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   mfamily                 36 non-null     object
 1   mgroup                  36 non-null     object
 2   mtype                   14 non-null     object
 3   origin_trade_number     36 non-null     int64 
 4   origin_contract_number  36 non-null     int64 
 5   maturity                36 non-null     object
dtypes: int64(2), object(4)
memory usage: 1.8+ KB


None

In [6]:
trade_details['maturity'] = pd.to_datetime(trade_details['maturity'])
trade_details_snapshot['maturity'] = pd.to_datetime(trade_details_snapshot['maturity'])

In [7]:
# we will set the index to origin_trade_number
trade_details = trade_details.set_index('origin_trade_number')
trade_details_snapshot = trade_details_snapshot.set_index("origin_trade_number")

In [8]:
# duplicated:
# mfamily = CURR
# mgroup = FXD
# mtype = SWLEG
# mismo origin_contract_number distinto origin_trade_number
# solo obtener la que tiene una mayor fecha de vencimiento
print(len(trade_details))
for col in trade_details.columns:
    print(col,len(trade_details[col].value_counts()))
    
    
print(len(trade_details_snapshot))
for col in trade_details_snapshot.columns:
    print(col,len(trade_details_snapshot[col].value_counts()))


31
mfamily 4
mgroup 7
mtype 4
origin_contract_number 26
maturity 13
36
mfamily 4
mgroup 7
mtype 4
origin_contract_number 31
maturity 18


In [9]:
# Paso 1: Filtrar las operaciones FXSwap
fxswap = trade_details[(trade_details['mfamily'] == 'CURR') |
                       (trade_details['mgroup'] == 'FXD') |
                       (trade_details['mtype'] == 'SWLEG')]
display(fxswap.head(3))
display(fxswap.info())

,mfamily,mgroup,mtype,origin_contract_number,maturity
origin_trade_number,,,,,
32734782,CURR,FXD,FXD,54853428,2021-09-22
19665185,CURR,FXD,SWLEG,18622136,2020-04-29
19665186,CURR,FXD,SWLEG,18622136,2020-12-30


<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 32734782 to 20665178
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   mfamily                 11 non-null     object        
 1   mgroup                  11 non-null     object        
 2   mtype                   11 non-null     object        
 3   origin_contract_number  11 non-null     int64         
 4   maturity                6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 528.0+ bytes


None

In [10]:
# we will fill missing values
null_maturity = fxswap[fxswap['maturity'].isnull()]
fxswap.loc[null_maturity.index,'maturity'] = trade_details_snapshot.loc[null_maturity.index, 'maturity'].head()

In [11]:
fxswap.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 32734782 to 20665178
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   mfamily                 11 non-null     object        
 1   mgroup                  11 non-null     object        
 2   mtype                   11 non-null     object        
 3   origin_contract_number  11 non-null     int64         
 4   maturity                11 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 828.0+ bytes


In [15]:
fxswap.head()

,mfamily,mgroup,mtype,origin_contract_number,maturity
origin_trade_number,,,,,
32734782,CURR,FXD,FXD,54853428,2021-09-22
19665185,CURR,FXD,SWLEG,18622136,2020-04-29
19665186,CURR,FXD,SWLEG,18622136,2020-12-30
19772399,CURR,FXD,SWLEG,18724280,2020-11-05
19772400,CURR,FXD,SWLEG,18724280,2021-11-05


In [39]:
to_delete = fxswap.groupby('origin_contract_number').agg({'maturity':'idxmin'})
to_delete = to_delete.reset_index()
to_delete = to_delete.rename(columns = {'maturity':'origin_trade_number'})
to_delete = to_delete.set_index('origin_trade_number')
print(to_delete)

                     origin_contract_number
origin_trade_number                        
19665185                           18622136
19772399                           18724280
20665177                           19622128
20980933                           19883451
22798004                           21622649
32734782                           54853428


In [40]:
final = trade_details.drop(index = to_delete.index)

In [41]:
display(final.head())
display(final.info())

,mfamily,mgroup,mtype,origin_contract_number,maturity
origin_trade_number,,,,,
316391872,IRD,BOND,NaN,678876251,2021-09-22
1111,IRD,LN_BR,NaN,2222,2022-10-06
2222222,IRD,IRS,NaN,2222222,2024-10-15
3815982,SCF,SCF,SCF,3672136,NaT
14596583,IRD,LN_BR,NaN,13774383,2020-12-29


<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 316391872 to 20665178
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   mfamily                 25 non-null     object        
 1   mgroup                  25 non-null     object        
 2   mtype                   8 non-null      object        
 3   origin_contract_number  25 non-null     int64         
 4   maturity                19 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.2+ KB


None

In [42]:
# save to csv
final.to_csv('challenge.csv')